### Situation 1 : singel prompting

In [ ]:
%pip install --upgrade openai

In [ ]:
from openai import OpenAI
import openai

API_KEY = "***********"

client = OpenAI(api_key=API_KEY)

# 調用API並獲得回覆
completion = client.chat.completions.create(
  # model="gpt-3.5-turbo"
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": """1. Extract all the characters, place names, time stamps, and key events from the provided text: “In a significant ruling ahead of the 2028 U.S. elections, a federal appeals court upheld a controversial Ohio law that shortened the state's early voting period and abolished the Golden Week. The decision overturned a May ruling by U.S. District Judge Michael Watson, who declared the law violated voters' rights. This forms part of a broader battle over state voting rules, including stricter voter identification laws and laws on felon voting rights. Critics, including civil rights groups, argue these changes limit minority participation.”.
2. Based on all the real information you know from the internet, assess the factualness of the extracted events. Show your analytic process.
3. Based on all the real information you know from the internet, assess the relationship between all characters, place names, time stamps, and key events. Show your analytic process.
4. Based on your analysis from steps 2 and 3, and all the information you currently have. Does this news article contain any misleading information or mismatched relationships? Show your analytic process and must respond with “YES” or “NO”.
5. How confident are you in your response, in terms of percentage? If your confidence is less than 50%, please change your answer to the opposite."""}],
    stream=False,
)

# 印出回覆
print(completion.choices[0].message.content)

### Situation 2 : recursive prompting

In [9]:
import pandas as pd
from openai import OpenAI

API_KEY = "*************"
client = OpenAI(api_key=API_KEY)

# 調用API並獲得回覆
def chatgpt_response(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return None

# read csv file
file_path = 'CoT_Prompt-Reuters-3.csv'  # hereeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 提取某列的内容並去掉缺失值
prompts = df['Reuters Version-Mix'].dropna().tolist()   # hereeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

# 建一個新的column來保存結果
output_df = pd.DataFrame(columns=df.columns.tolist() + ['ChatGPT Response'])

# 獲得GPT回覆並逐行保存到CSV
output_file_path = 'ChatGPT_Responses-Reuters-3-4o.csv' # hereeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
for i, prompt in enumerate(prompts):
    response = chatgpt_response(prompt)
    new_row = df.iloc[i].tolist() + [response]
    output_df.loc[i] = new_row
    output_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Processed all prompts and saved responses to {output_file_path}")


Processed all prompts and saved responses to ChatGPT_Responses-test.csv
